<b>теги UT <– теги pymorphy2</b><br>
ADJ: adjective <– ADJF	имя прилагательное (полное)	хороший; ADJS	имя прилагательное (краткое)	хорош<br>
ADP: adposition <– PREP	предлог	в<br>
ADV: adverb <– ADVB	наречие	круто<br>
AUX: auxiliary <–<br>
CCONJ: coordinating conjunction <– <br>
DET: determiner <–<br>
INTJ: interjection <– INTJ<br>
NOUN: noun <– NOUN	имя существительное	хомяк<br>
NUM: numeral <– NUMR	числительное	три, пятьдесят<br>
PART: particle <– PRCL	частица	бы, же, лишь<br>
PRON: pronoun <– NPRO	местоимение-существительное	он<br>
PROPN: proper noun <–<br>
PUNCT: punctuation <–<br>
SCONJ: subordinating conjunction <–<br>
SYM: symbol <–<br>
VERB: verb <– VERB	глагол (личная форма)	говорю, говорит, говорил<br>
X: other <–<br>
<br>
COMP	компаратив	лучше, получше, выше<br>
PRTF	причастие (полное)	прочитавший, прочитанная<br>
PRTS	причастие (краткое)	прочитана<br>
GRND	деепричастие	прочитав, рассказывая<br>
PRED	предикатив	некогда<br>
CONJ	союз	и

То, что не совпадает и для чего нельзя установить точное соответствие, учитывать не будем.



In [1]:
import sys
import gensim, logging
from nltk.corpus import stopwords
from gensim.models.doc2vec import LabeledSentence
from gensim.models import Doc2Vec
from sklearn.linear_model import LogisticRegression
import re
import os
import pymorphy2
import numpy as np


logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)
stop_words = set(stopwords.words('russian'))
morph = pymorphy2.MorphAnalyzer()
RE_POS = re.compile('^....')
RE_WORDS = re.compile('[\w]+')



def lemmatize(path):
    new_text = ''
    with open(path, 'r', encoding='utf-8') as f:
        text = f.read()
        words = RE_WORDS.findall(text)
        words = [word for word in words if len(word) > 0 and word not in stop_words]
        for word in words:
            analysis = morph.parse(word)[0]
            lemma = analysis.normal_form
            p = RE_POS.search(str(analysis.tag))
            if p:
                pos = p.group()
                if pos == 'ADJF' or pos == 'ADJS':
                    pos = 'ADJ'
                elif pos == 'PREP':
                    pos = 'ADP'
                elif pos == 'ADVB':
                    pos = 'ADV'
                elif pos == 'NUMR':
                    pos = 'NUM'
                elif pos == 'PRCL':
                    pos = 'PART'
                elif pos == 'NPRO':
                    pos = 'PRON'
                elif pos == 'CONJ': # but this is not true
                    pos = 'CCONJ'
            else:
                print('no pos detected')
            new_word = '{}_{}\n'.format(lemma, pos)
            new_text += new_word
    with open('lem_' + path, 'w', encoding='utf-8') as f:
        f.write(new_text)

2017-03-30 20:01:56,005 : INFO : Loading dictionaries from /Library/Frameworks/Python.framework/Versions/3.5/lib/python3.5/site-packages/pymorphy2_dicts/data
2017-03-30 20:01:56,068 : INFO : format: 2.4, revision: 393442, updated: 2015-01-17T16:03:56.586168


In [2]:
for root, dirs, files in os.walk('texts'):
    for file_name in files:
        if file_name.endswith('.txt'):
            file_path = os.path.join(root, file_name)
            lemmatize(file_path)

In [3]:
m = 'ruscorpora_1_300_10.bin.gz'
model = gensim.models.KeyedVectors.load_word2vec_format(m, binary=True)
model.init_sims(replace=True)

2017-03-30 20:03:51,415 : INFO : loading projection weights from ruscorpora_1_300_10.bin.gz
2017-03-30 20:04:02,179 : INFO : loaded (184973, 300) matrix from ruscorpora_1_300_10.bin.gz
2017-03-30 20:04:02,180 : INFO : precomputing L2-norms of word weight vectors


In [4]:
text_vectors = []
for root, dirs, files in os.walk('lem_texts'):
    for file_name in files:
        if file_name.endswith('.txt'):
            word_vectors = []
            file_path = os.path.join(root, file_name)
            with open(file_path, 'r', encoding='utf-8') as f:
                words = f.read().split('\n')
                for word in words:
                    try:
                        word_vectors.append(model[word])
                    except KeyError:
                        pass
            text_vectors.append(np.mean(np.array(word_vectors), axis=0))

Вектора для текстов:

In [5]:
text_vectors[:10]

[array([ -1.17968842e-02,   1.13915270e-02,   2.35911701e-02,
          1.67501699e-02,  -1.48474704e-02,   3.50735360e-03,
          1.17540127e-02,   1.26293106e-02,   4.11255239e-03,
         -2.24847812e-02,   6.25815289e-03,  -2.68820580e-02,
          1.27218729e-02,  -1.73662425e-04,   7.95563031e-03,
         -1.44793745e-02,   2.88764313e-02,  -5.60187595e-03,
          2.03351192e-02,  -5.18632401e-03,   3.74118872e-02,
         -4.24738682e-04,  -1.78041495e-02,   2.94190478e-02,
         -1.71062490e-03,  -2.22654063e-02,   2.36980636e-02,
         -2.92405728e-02,  -1.79384090e-02,  -2.69630700e-02,
          9.17280372e-03,   2.19869893e-02,   4.59149666e-02,
         -1.62656438e-02,   1.27132609e-02,   1.17297098e-03,
          8.31236690e-03,   1.35489937e-03,  -1.80331450e-02,
         -8.93882243e-04,   2.83783767e-02,   1.44638773e-02,
          2.07680203e-02,  -1.44070247e-02,   4.38668532e-03,
          4.72331280e-03,   3.68453600e-02,   1.45343468e-02,
        

Вектор классов:

In [9]:
#classes = np.array([[1]] * 125 + [[2]] * 125 + [[3]] * 125)

classes = np.array([1] * 125 + [2] * 125 + [3] * 125)
text_vect = np.array(text_vectors)
#data = np.hstack((text_vect, classes))
#text_vect[:10]

In [7]:
classes[:10]

array([1, 1, 1, 1, 1, 1, 1, 1, 1, 1])

Делаем логит:

In [16]:
from sklearn.model_selection import cross_val_score


clf = LogisticRegression(penalty="l2", solver="lbfgs", multi_class="multinomial", max_iter=300, n_jobs=1)


По метрике accuracy:

In [20]:
scores = cross_val_score(clf, text_vect, classes, cv=5, scoring='accuracy')
#clf.fit(text_vect, classes)
print(scores)
np.mean(scores)

[ 0.88        0.85333333  0.90666667  0.90666667  0.84      ]


0.8773333333333333

По F-мере:

In [22]:
scores = cross_val_score(clf, text_vect, classes, cv=5, scoring='f1_macro')
print(scores)
np.mean(scores)

[ 0.87982906  0.85219199  0.90604575  0.90523969  0.8352415 ]


0.87570959798021342